In [14]:
!pip install cryptography
import random
import math
import binascii
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
import secrets
!pip install sympy
from sympy import mod_inverse
from sympy import isprime as isPrime
import hashlib
import hmac



In [15]:
class seedGenerator:
  def seed_gen():
    return secrets.randbelow(10**25 - 10**16) + 10**16

  def get_middle_digits(seed: int):
    seed_str = str(seed)
    length = len(seed_str)

    if length < 15:
        return seed

    start = (length // 2) - 7
    end = (length // 2) + 8

    return int(seed_str[start:end])


  def iteration(x: int, seed: int):
    current = seed

    for i in range(x):
        middle = seedGenerator.get_middle_digits(current)
        current = middle ** 2

    return current


creation of private key of bob and sending the seed to alice so itll be shared

In [16]:
# Cell 3: RSA Helper Functions and Seed Exchange Logic


def miller_rabin(n, k=5):
    if n == 2 or n == 3:
        return True
    if n < 2 or n % 2 == 0:
        return False

    d, r = n - 1, 0
    while d % 2 == 0:
        d //= 2
        r += 1

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def mod_inverse(a, m):
    m0, x0, x1 = m, 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += m0
    return x1

def generate_prime():
    while True:
        candidate = secrets.randbits(514)
        candidate |= 1
        if miller_rabin(candidate):
            return candidate

def gen_rsa_bob_keys():
    print("Generating Bob's RSA primes (p and q)... This might take a moment.")
    p = generate_prime()
    q = generate_prime()

    while p == q:
        q = generate_prime()

    N = p * q
    phi_N = (p - 1) * (q - 1)

    e = 65537
    d = mod_inverse(e, phi_N)

    public_key = (e, N)
    private_key = (d, N)

    return public_key, private_key

def encrypt(seed_val, public_key):
    e, N = public_key
    return pow(seed_val, e, N)

def decrypt(cipherText, private_key):
    d, n = private_key
    return pow(cipherText, d, n)

def exchange_seeds():
    print("\n--- Initiating Seed Exchange (RSA) ---")
    bob_public_key, bob_private_key = gen_rsa_bob_keys()
    print("Bob's RSA keys generated.")

    # This line calls seedGenerator.seed_gen() from Cell 2
    alice_seed = seedGenerator.seed_gen()
    print(f"Alice generates a random seed: {alice_seed}")

    alice_cipherText = encrypt(alice_seed, bob_public_key)
    print(f"Alice encrypts seed for Bob: {alice_cipherText}")

    decrypted_bob_seed = decrypt(alice_cipherText, bob_private_key)
    print(f"Bob decrypts seed: {decrypted_bob_seed}")

    if alice_seed == decrypted_bob_seed:
        print("Seed successfully exchanged and verified!")
    else:
        print("There was an error in the seed exchange!")

    # This function returns the seeds and Bob's keys
    return alice_seed, decrypted_bob_seed, bob_public_key, bob_private_key



alice_seed, bob_received_seed, bob_public_key_rsa, bob_private_key_rsa = exchange_seeds()




--- Initiating Seed Exchange (RSA) ---
Generating Bob's RSA primes (p and q)... This might take a moment.
Bob's RSA keys generated.
Alice generates a random seed: 4630329492838465589089189
Alice encrypts seed for Bob: 1082376598402334935774054687823869057460229249448182329422877854781822579382865708683386742343746089944636470525418057785952424549419785131237359025217208477151362542002536272483057610510176636644958785250023825684353342667342570791193007646795521941202959054032789801361850282929337744011159441126961119066814
Bob decrypts seed: 4630329492838465589089189
Seed successfully exchanged and verified!


bobs keys:
bob_private_key (e,d)
public_key1 (e,n)

In [17]:
def gen_rsa_alice_keys():
  p = generate_prime()
  q = generate_prime()

  while p==q:
    q = generate_prime()

  N = p*q
  phi_N = (p-1)*(q-1)

  e = 65539
  d = mod_inverse (e,phi_N)

  public_key = (e,N)
  private_key = (d, N)

  return public_key,private_key




alice keys
alice_private_key
alice_public_key

In [18]:



ITER_BITS = 16
LEN_BITS = 16

def encrypt_message(message, seedGenerator, seed):
    def to_binary(text):
        return ''.join([bin(ord(char))[2:].zfill(8) for char in text])

    message_binary = to_binary(message)
    original_length_in_bits = len(message_binary)

    iteration = secrets.randbelow(9000) + 1000

    embedded_iteration_binary = bin(iteration)[2:].zfill(ITER_BITS)
    embedded_length_binary = bin(original_length_in_bits)[2:].zfill(LEN_BITS)

    combined_plain_binary = embedded_length_binary + message_binary + embedded_iteration_binary

    cipher_num = seedGenerator.iteration(iteration, seed)
    cipher_binary = bin(cipher_num)[2:]

    if len(cipher_binary) < len(combined_plain_binary):
        cipher_binary = (cipher_binary * (len(combined_plain_binary) // len(cipher_binary) + 1))[:len(combined_plain_binary)]

    def xor_binary(data_binary, key_stream_binary):
        return ''.join('1' if b1 != b2 else '0' for b1, b2 in zip(data_binary, key_stream_binary))

    encrypted_combined_binary = xor_binary(combined_plain_binary, cipher_binary)

    def binary_to_ascii_string(binary_data):
        if len(binary_data) % 8 != 0:
             binary_data += '0' * (8 - (len(binary_data) % 8))
        chunks = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
        ascii_chars = [chr(int(chunk, 2)) for chunk in chunks]
        return ''.join(ascii_chars)

    final_encrypted_ascii_payload = binary_to_ascii_string(encrypted_combined_binary)

    # --- NEW: Generate HMAC-SHA256 Authentication Tag ---
    # Derive a consistent HMAC key from the shared seed.
    # We convert the integer seed to a string and then to bytes, as HMAC requires a bytes key.
    # In a real system, you'd use a more robust Key Derivation Function (KDF) here.
    hmac_key = str(seed).encode('utf-8')

    # Calculate the HMAC-SHA256 of the encrypted payload using the derived key.
    # The payload also needs to be converted to bytes for HMAC.
    authentication_tag = hmac.new(hmac_key, final_encrypted_ascii_payload.encode('utf-8'), hashlib.sha256).hexdigest()
    # ---------------------------------------------------

    # Return the encrypted payload, the iteration, the original length, AND the authentication tag.
    return final_encrypted_ascii_payload, iteration, original_length_in_bits, authentication_tag

encrypting first layer of bob message

In [19]:


def decrypt_message(encrypted_ascii_string, received_iteration_for_key_gen, received_length_for_final_trim, seed, received_authentication_tag):
    # --- NEW: HMAC-SHA256 Authentication Tag Verification ---
    # Derive the SAME HMAC key using the shared seed that the sender used.
    hmac_key = str(seed).encode('utf-8')

    # Recalculate the HMAC-SHA256 of the received encrypted string.
    recalculated_tag = hmac.new(hmac_key, encrypted_ascii_string.encode('utf-8'), hashlib.sha256).hexdigest()

    # Compare the recalculated tag with the received tag.
    # hmac.compare_digest() is used for constant-time comparison to prevent timing attacks.
    if not hmac.compare_digest(recalculated_tag, received_authentication_tag):
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("ATTENTION: MESSAGE TAMPERED OR FORGED! HMAC MISMATCH!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return "ERROR: Message Tampered or Forged!" # Reject the message if integrity is compromised
    else:
        print("Message Integrity and Authenticity OK. Proceeding with decryption.")
    # -----------------------------------------------------------------

    # 1. Convert encrypted ASCII string back to binary
    def ascii_string_to_binary(ascii_str):
       return ''.join([bin(ord(c))[2:].zfill(8) for c in ascii_str])

    encrypted_combined_binary = ascii_string_to_binary(encrypted_ascii_string)

    # 2. Re-generate the key stream using the EXTERNALLY RECEIVED iteration and shared seed
    cipher_num = seedGenerator.iteration(received_iteration_for_key_gen, seed)
    cipher_binary = bin(cipher_num)[2:]

    if len(cipher_binary) < len(encrypted_combined_binary):
        cipher_binary = (cipher_binary * (len(encrypted_combined_binary) // len(cipher_binary) + 1))[:len(encrypted_combined_binary)]

    def xor_binary(data_binary, key_stream_binary):
        return ''.join('1' if b1 != b2 else '0' for b1, b2 in zip(data_binary, key_stream_binary))

    decrypted_plain_payload_binary = xor_binary(encrypted_combined_binary, cipher_binary)

    # 4. Parse the decrypted plain payload to extract embedded metadata and original message
    extracted_embedded_iteration_binary = decrypted_plain_payload_binary[-ITER_BITS:]
    extracted_embedded_length_binary = decrypted_plain_payload_binary[:LEN_BITS]

    original_message_binary = decrypted_plain_payload_binary[LEN_BITS : -ITER_BITS]

    recovered_embedded_iteration = int(extracted_embedded_iteration_binary, 2)
    recovered_embedded_length = int(extracted_embedded_length_binary, 2)

    def binary_to_text(binary_data, final_msg_length_in_bits):
        trimmed_binary_data = binary_data[:final_msg_length_in_bits]
        chars = [chr(int(trimmed_binary_data[i:i+8], 2)) for i in range(0, len(trimmed_binary_data), 8)]
        return ''.join(chars)

    original_message = binary_to_text(original_message_binary, recovered_embedded_length)

    return original_message

In [20]:
def rsa_encrypt(message: str, public_key: tuple):
    e, N = public_key
    message_bytes = message.encode('utf-8')
    message_int = int.from_bytes(message_bytes, 'big')
    rsa_encrypted = pow(message_int, e, N)
    return rsa_encrypted


def rsa_decrypt(encrypted_message: int, private_key: tuple):
    d, N = private_key
    decrypted_int = pow(encrypted_message, d, N)
    decrypted_bytes = decrypted_int.to_bytes((decrypted_int.bit_length() + 7) // 8, 'big')
    decrypted_str = decrypted_bytes.decode('utf-8')
    return decrypted_str


In [ ]:

print("\n--- Starting Encrypted Chat Simulation ---")
print("Type 'exit' at any prompt to end the chat.")

while True:
    # --- Alice sends a message to Bob ---
    alice_msg = input("\nAlice (Type your message): ")
    if alice_msg.lower() == 'exit':
        print("Chat ended.")
        break

    # Encrypt Alice's message, including generating an HMAC tag
    final_encrypted_ascii_payload_alice, iteration_alice, original_length_alice, tag_alice = encrypt_message(alice_msg, seedGenerator, alice_seed)

    print(f"\n--- Alice to Bob (Encrypted & Authenticated) ---")
    print(f"Payload: {final_encrypted_ascii_payload_alice}")
    print(f"HMAC Tag: {tag_alice}")

    # Simulate Bob receiving and decrypting
    received_payload_by_bob = final_encrypted_ascii_payload_alice
    received_iteration_by_bob = iteration_alice
    received_length_by_bob = original_length_alice
    received_tag_by_bob = tag_alice

    decrypted_msg_bob = decrypt_message(
        received_payload_by_bob,
        received_iteration_by_bob,
        received_length_by_bob,
        bob_received_seed,
        received_tag_by_bob
    )
    print(f"Bob decrypted: {decrypted_msg_bob}")


    # --- Bob sends a reply to Alice ---
    bob_msg = input("\nBob (Type your reply): ")
    if bob_msg.lower() == 'exit':
        print("Chat ended.")
        break

    # Encrypt Bob's reply, including generating an HMAC tag
    final_encrypted_ascii_payload_bob, iteration_bob, original_length_bob, tag_bob = encrypt_message(bob_msg, seedGenerator, bob_received_seed)

    print(f"\n--- Bob to Alice (Encrypted & Authenticated) ---")
    print(f"Payload: {final_encrypted_ascii_payload_bob}")
    print(f"HMAC Tag: {tag_bob}")

    # Simulate Alice receiving and decrypting
    received_payload_by_alice = final_encrypted_ascii_payload_bob
    received_iteration_by_alice = iteration_bob
    received_length_by_alice = original_length_bob
    received_tag_by_alice = tag_bob

    decrypted_msg_alice = decrypt_message(
        received_payload_by_alice,
        received_iteration_by_alice,
        received_length_by_alice,
        alice_seed,
        received_tag_by_alice
    )
    print(f"Alice decrypted: {decrypted_msg_alice}")


--- Starting Encrypted Chat Simulation ---
Type 'exit' at any prompt to end the chat.

Alice (Type your message): hi!

--- Alice to Bob (Encrypted & Authenticated) ---
Payload: àAè · 
HMAC Tag: 66f0e1e511ab165d8055a5782069f3184b7b7f135a60911acbf8bd6c9e0a4b30
Message Integrity and Authenticity OK. Proceeding with decryption.
Bob decrypted: hi!

Bob (Type your reply): hello

--- Bob to Alice (Encrypted & Authenticated) ---
Payload: ¦ðæåò
HMAC Tag: f965f6610ccf832f2dde2795ad3fbd0a55ecb575be545f6cdb48fefaa950c3a2
Message Integrity and Authenticity OK. Proceeding with decryption.
Alice decrypted: hello
